In [77]:
import random
import copy

class TabuSearch:
    def __init__(self, set_X, subsets, max_iterations=1000, tabu_tenure=10):
        self.set_X = set_X
        self.subsets = subsets
        self.max_iterations = max_iterations
        self.tabu_tenure = tabu_tenure
        self.tabu_list = []
        self.best_solution = None
        self.best_cost = float('inf')
    
    def cost_function(self, solution):
        covered_elements = set()
        for subset in solution:
            covered_elements.update(subset)
        uncovered_elements = set(self.set_X) - covered_elements
        return len(uncovered_elements)
    
    def generate_initial_solution(self):
        # Select subsets that greedily cover the most uncovered elements
        solution = []
        covered_elements = set()
        remaining_elements = set(self.set_X)

        while remaining_elements:
            best_subset = None
            best_coverage = 0
            for subset in self.subsets:
                coverage = len(set(subset) & remaining_elements)
                if coverage > best_coverage:
                    best_subset = subset
                    best_coverage = coverage
            
            if best_subset:
                solution.append(best_subset)
                covered_elements.update(best_subset)
                remaining_elements -= set(best_subset)
            else:
                break

        return solution
    
    def generate_neighbors(self, solution):
        neighbors = []
        # Explore neighbors by swapping, adding, or removing subsets
        for i in range(len(solution)):
            for subset in self.subsets:
                if subset not in solution:
                    new_solution = copy.deepcopy(solution)
                    new_solution[i] = subset
                    if self.is_valid_solution(new_solution):
                        neighbors.append(new_solution)

        # Try adding a new subset that could complete the solution
        for subset in self.subsets:
            if subset not in solution and self.is_valid_addition(solution, subset):
                new_solution = copy.deepcopy(solution)
                new_solution.append(subset)
                if self.is_valid_solution(new_solution):
                    neighbors.append(new_solution)

        return neighbors
    
    def is_valid_solution(self, solution):
        covered_elements = set()
        for subset in solution:
            if covered_elements.intersection(set(subset)):
                return False  # Overlap found
            covered_elements.update(subset)
        return True
    
    def is_valid_addition(self, solution, new_subset):
        covered_elements = set()
        for subset in solution:
            covered_elements.update(subset)
        return not covered_elements.intersection(set(new_subset))
    
    def tabu_search(self):
        current_solution = self.generate_initial_solution()
        current_cost = self.cost_function(current_solution)
        
        for iteration in range(self.max_iterations):
            neighbors = self.generate_neighbors(current_solution)
            best_neighbor = None
            best_neighbor_cost = float('inf')
            
            for neighbor in neighbors:
                if neighbor not in self.tabu_list:
                    cost = self.cost_function(neighbor)
                    if cost < best_neighbor_cost:
                        best_neighbor = neighbor
                        best_neighbor_cost = cost
            
            if best_neighbor:
                current_solution = best_neighbor
                current_cost = best_neighbor_cost
                self.tabu_list.append(current_solution)
                if len(self.tabu_list) > self.tabu_tenure:
                    self.tabu_list.pop(0)
                
                if current_cost < self.best_cost and self.is_valid_solution(current_solution):
                    self.best_solution = current_solution
                    self.best_cost = current_cost
                    
            if current_cost == 0:  # Perfect solution found
                break
        
        return self.best_solution, self.best_cost
    
def read_benchmark_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        set_X = list(map(int, lines[0].split()))
        num_subsets = int(lines[1].strip())
        subsets = [list(map(int, line.split())) for line in lines[2:num_subsets+2]]
        return set_X, subsets

if __name__ == "__main__":
    # Example usage
    file_path = "C:/Users/abode/OneDrive/Desktop/Benchmarks/Benchmarks/Test.txt"
    set_X, subsets = read_benchmark_file(file_path)
    
    tabu_search_solver = TabuSearch(set_X, subsets)
    solution, cost = tabu_search_solver.tabu_search()
    
    if cost == 0:
        print("Exact solution found:", solution)
    else:
        print("No exact solution found. Best attempt:", solution)


Exact solution found: [[1, 2, 3, 4], [5, 6, 7, 8]]
